In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from math import exp, sqrt

In [2]:
# Parameters
# ----------
p_air = 1.e5
alpha = 0.06
m = 9.
b = 2000.
ca = 355e-6 * p_air
d_leaf = 0.04
vmax_25 = 69.

In [3]:
def esat(temp_deg):
    return 100.*6.112*exp(17.62*temp_deg/(243.12 + temp_deg))

In [4]:
def photosynthesis(pathway, ci, par, vmax, temp_leaf):
    
    kc = 30. * 2.1**((temp_leaf - 25.)/10.)
    ko = 30000. * 1.2**((temp_leaf - 25.)/10.)
    oi = 0.209*p_air
    gamma_star = 0.5 * kc/ko * 0.21*oi
    
    assert ci - gamma_star >= 0.
    
    if pathway == 'C3':
        wc = (ci - gamma_star)*vmax / (ci + kc*(1.+oi/ko))
        wj = (ci - gamma_star)*4.6*par*alpha / (ci + 2.*gamma_star)
    elif pathway == 'C4':
        wc = vmax
        wj = 4.6*par*alpha
    else:
        "WARNING! PHOTOSYNTHETIC PATHWAY WRONGLY SPECIFIED"
    
    # Gross leaf photosynthesis
    ag = min(wc,wj)
    return ag

In [5]:
def phot_thermal_breakdown(temp_leaf):
    return 1. / (1. + exp((-220000.+710.*(temp_leaf+273.15))/(8314.4676*0.001*(temp_leaf+273.15))))

In [6]:
def lpj_thermal_inhibition(temp_leaf):
    pstemp_min = -5
    pstemp_low = 10
    pstemp_high = 30
    pstemp_max = 45
    k1 = (pstemp_min+pstemp_low) / 2.0;
    return (1. - .01*exp(4.6/(pstemp_max-pstemp_high)*(temp_leaf-pstemp_high)))/(1.0+exp((k1-temp_leaf)/(k1-pstemp_min)*4.6));

In [7]:
def stomatal(pathway, par, beta, temp_leaf, rh_air, temp_air=20, wind_speed=10):
    
    ea = esat(temp_air)*rh_air/100.
    ptemp_air = (temp_air+273.15)*(p_air/1.e5)**0.28557 # Potential temperature of air
    co2m = 1.e-9 * 8314.4676 * ptemp_air/p_air # [umol-1 m2 s / s m-1]
    rb = 223.*sqrt(d_leaf/wind_speed) * co2m
    
    vmax = vmax_25 * 2.4**((temp_leaf - 25.)/10.) * beta * phot_thermal_breakdown(temp_leaf)

    
    if pathway == 'C3':
        ci = 0.7*ca
    elif pathway == 'C4':
        ci = 0.4*ca
    else:
        "WARNING! PHOTOSYNTHETIC PATHWAY WRONGLY SPECIFIED"
    
    for _ in range(3):
        
        ag = photosynthesis(pathway, ci, par, vmax, temp_leaf)
        
        cs = ca - 1.37*rb*p_air*ag
        ei = esat(temp_leaf)
        eap = max(min(ea, ei),0.25*ei)
        
        aaa = m*ag*p_air*eap/cs/ei + b
        bbb = m*ag*p_air*rb/cs + b*rb - 1.
        ccc = -rb
        
        rs1 = (-bbb + sqrt(bbb**2 - 4.*aaa*ccc)) / (2.*aaa)
        rs2 = (-bbb - sqrt(bbb**2 - 4.*aaa*ccc)) / (2.*aaa)
        rs = max(rs1, rs2)
        
        lambda_max = 0.8
        ci = min(cs - 1.65*rs*p_air*ag, lambda_max*ca)
    
    rs = rs/co2m
    
    print("\n")
    print("Photon flux [10^(-6) mol / m^2 / s] = " + str(round(4.6*par,3)))
    print("Gross leaf photosynthesis [10^(-6) mol / m^2 / s] = " + str(round(ag,3)))
    print("Stomatal resistance [s/m] = " + str(round(rs,3)))
    
    return

In [8]:
interact(stomatal,
         pathway = widgets.RadioButtons(options=['C3', 'C4'], value='C3',
                                         description='Photosynthetic pathway:',
                                         disabled=False),
          par = widgets.IntSlider(min=0,max=200,step=1,value=50,description="PAR [W/m2]"),
          beta = widgets.FloatSlider(min=0,max=1,step=0.05,value=0.75,description="Soil moisture"),
          temp_leaf = widgets.IntSlider(min=-20,max=40,step=1,value=15,description="Leaf temp."),
          rh_air = widgets.FloatSlider(min=0,max=100,step=5,value=50,description="Air humidity"),
          temp_air = widgets.IntSlider(min=3,max=30,step=1,value=12,description="Air temp."),
          wind_speed = widgets.FloatSlider(min=0.1,max=10,step=0.1,value=1.,description="Wind speed"),
         )

interactive(children=(RadioButtons(description='Photosynthetic pathway:', options=('C3', 'C4'), value='C3'), I…

<function __main__.stomatal(pathway, par, beta, temp_leaf, rh_air, temp_air=20, wind_speed=10)>